In [ ]:
import whisper
import subprocess
import os
import re

# путь к твоему видео
video_path = r"VIDEO\What is ORM Entity Framework Code-First in .NET Create Tables in.mp4"
audio_path = "temp_audio.wav"
output_text_file = "output_for_tts.txt"

# --- 1. Извлекаем аудио из видео ---
subprocess.run([
    "ffmpeg", "-i", video_path,
    "-vn",                 # без видео
    "-acodec", "pcm_s16le",
    "-ar", "16000",
    "-ac", "1",
    audio_path,
    "-y"
])

# --- 2. Загружаем модель Whisper ---
model = whisper.load_model("medium")

# --- 3. Распознаем и переводим ---
result = model.transcribe(
    audio_path,
    task="translate",      # переводим в русский текст
    language="en"          # язык исходного видео
)

raw_text = result["text"]

# --- 4. Очищаем текст — убираем мусор ---
clean_text = raw_text.strip()
clean_text = re.sub(r"\s+", " ", clean_text)  # убираем лишние пробелы

# --- 5. Сохраняем в TXT ---
with open(output_text_file, "w", encoding="utf-8") as f:
    f.write(clean_text)

print("\nФайл успешно сохранён:")
print(output_text_file)

# удалить временный аудиофайл (по желанию)
os.remove(audio_path)


c:\Users\AKULA\anaconda3\envs\GPU\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
c


Файл успешно сохранён:
output_for_tts.txt


In [6]:
!pip install deep-translator


   ---------------------------------------- 0.0/42.3 kB ? eta -:--:--
   ---------------------------------------- 42.3/42.3 kB 1.0 MB/s eta 0:00:00


In [2]:
from deep_translator import GoogleTranslator

# === файлы ===
input_file = "output_for_tts.txt"
output_file = "output_ru.txt"

# === читаем текст ===
with open(input_file, "r", encoding="utf-8") as f:
    text = f.read().strip()

# === функция для разбиения текста ===
def split_text(text, max_len=4000):
    parts = []
    while len(text) > max_len:
        # ищем ближайшую точку, чтобы не рвать предложения
        cut = text.rfind(".", 0, max_len)
        if cut == -1:
            cut = max_len
        parts.append(text[:cut+1].strip())
        text = text[cut+1:].strip()
    parts.append(text)
    return parts

chunks = split_text(text, 4000)

print(f"Найдено частей: {len(chunks)}")

translated_parts = []

# === переводим по частям ===
for i, chunk in enumerate(chunks, 1):
    print(f"Перевод части {i}/{len(chunks)}...")
    translated = GoogleTranslator(source="auto", target="ru").translate(chunk)
    translated_parts.append(translated)

# === сохраняем в файл ===
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n\n".join(translated_parts))

print("\nГотово! Переведённый файл сохранён как:", output_file)


Найдено частей: 8
Перевод части 1/8...
Перевод части 2/8...
Перевод части 3/8...
Перевод части 4/8...
Перевод части 5/8...
Перевод части 6/8...
Перевод части 7/8...
Перевод части 8/8...

Готово! Переведённый файл сохранён как: output_ru.txt


In [ ]:
# дальше через ии

In [ ]:
import subprocess
from moviepy.editor import VideoFileClip, ColorClip, concatenate_videoclips

video_path = r"VIDEO\Entity Framework Core Crash Course C .NET EF Core Tutorial for B.mp4"
audio_path = "output_final_clean.wav"
output_path = "final_video.mp4"


clip = VideoFileClip(video_path).subclip(10)  # пропускаем первые 10 секунд
clip.write_videofile(output_path, codec="libx264")
video_clip = clip
video_duration = video_clip.duration

# 2️⃣ Получаем длительность аудио
import wave

with wave.open(audio_path, "r") as wav_file:
    frames = wav_file.getnframes()
    rate = wav_file.getframerate()
    audio_duration = frames / float(rate)

print(f"Видео длительность: {video_duration}s, Аудио длительность: {audio_duration}s")

# 3️⃣ Добавляем черный экран, если аудио длиннее видео
if audio_duration > video_duration:
    extra_duration = audio_duration - video_duration
    black_clip = ColorClip(size=video_clip.size, color=(0,0,0), duration=extra_duration)
    video_clip = concatenate_videoclips([video_clip, black_clip])

# 4️⃣ Сохраняем видео без звука во временный файл
temp_video = "temp_video_no_audio.mp4"
video_clip.write_videofile(temp_video, audio=False, codec="libx264", fps=video_clip.fps)

# 5️⃣ Используем ffmpeg для замены аудио
subprocess.run([
    "ffmpeg",
    "-y",  # перезаписать если есть
    "-i", temp_video,
    "-i", audio_path,
    "-c:v", "copy",
    "-c:a", "aac",
    "-b:a", "192k",
    "-shortest",  # чтобы видео длилось по меньшей длительности
    output_path
])

print("Готово! Итоговое видео:", output_path)


Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_video.mp4



Moviepy - Done !
Moviepy - video ready final_video.mp4
Видео длительность: 11248.93s, Аудио длительность: 10725.689333333334s
Moviepy - Building video temp_video_no_audio.mp4.
Moviepy - Writing video temp_video_no_audio.mp4



Moviepy - Done !
Moviepy - video ready temp_video_no_audio.mp4
Готово! Итоговое видео: final_video.mp4
